In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import MNIST, ImageFolder

from tqdm.notebook import tqdm

In [2]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
PATH_TO_SAVE_MNIST = '../dataset/'
PATH_IMAGE_TRAIN = '../images/train'
PATH_IMAGE_TEST = './images'

In [4]:
batch_size_train = 64
batch_size_val = 1024

In [5]:
transform_MNIST = T.Compose([
                    T.ToTensor(),
                ])

transform = T.Compose([
                T.ToTensor(),
                T.Grayscale(),
                T.Resize((28, 28)),
            ])

train_dataset = MNIST(PATH_TO_SAVE_MNIST, train=True, download=True, transform=transform_MNIST)
val_dataset = MNIST(PATH_TO_SAVE_MNIST, train=False, download=True, transform=transform_MNIST)

test_dataset = ImageFolder(root=PATH_IMAGE_TEST, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

test_loader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=3)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #input channel 1, output channel 10
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        #input channel 10, output channel 20
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        #dropout layer
        self.conv2_drop = nn.Dropout2d()
        #fully connected layer
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        counter += 1
        tk0.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def make_pred(model, device, test_loader):
    model.eval()
    i = 0
    for data, _ in test_loader:
        data = data.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        print(pred)
        print(max(pred))
        if i > 9: break
        i += 1


In [8]:
from torch.optim import Adam

learning_rate = 0.001
model = CNN().to(DEVICE)
optimizer = Adam(model.parameters(), lr=learning_rate)

num_epoch = 10
for epoch in range(1, num_epoch + 1):
        train(model, DEVICE, train_loader, optimizer, epoch)
        test(model, DEVICE, val_loader)


Test set: Average loss: 0.2214, Accuracy: 9344/10000 (93%)



Test set: Average loss: 0.1643, Accuracy: 9500/10000 (95%)



Test set: Average loss: 0.1277, Accuracy: 9639/10000 (96%)



Test set: Average loss: 0.1132, Accuracy: 9685/10000 (97%)



Test set: Average loss: 0.1034, Accuracy: 9700/10000 (97%)



Test set: Average loss: 0.0994, Accuracy: 9722/10000 (97%)



Test set: Average loss: 0.0951, Accuracy: 9717/10000 (97%)



Test set: Average loss: 0.0875, Accuracy: 9741/10000 (97%)



Test set: Average loss: 0.0827, Accuracy: 9767/10000 (98%)



Test set: Average loss: 0.0853, Accuracy: 9746/10000 (97%)


In [9]:
make_pred(model, DEVICE, test_loader)

tensor([[1],
        [7],
        [3]], device='cuda:0')
tensor([7], device='cuda:0')
tensor([[2],
        [3],
        [0]], device='cuda:0')
tensor([3], device='cuda:0')
tensor([[8],
        [1],
        [3]], device='cuda:0')
tensor([8], device='cuda:0')
tensor([[1],
        [3],
        [5]], device='cuda:0')
tensor([5], device='cuda:0')
tensor([[8],
        [2],
        [5]], device='cuda:0')
tensor([8], device='cuda:0')
tensor([[0],
        [2],
        [3]], device='cuda:0')
tensor([3], device='cuda:0')
tensor([[3],
        [2],
        [4]], device='cuda:0')
tensor([4], device='cuda:0')
tensor([[4],
        [6],
        [6]], device='cuda:0')
tensor([6], device='cuda:0')
tensor([[5],
        [6],
        [2]], device='cuda:0')
tensor([6], device='cuda:0')
tensor([[6],
        [6],
        [2]], device='cuda:0')
tensor([6], device='cuda:0')
tensor([[5],
        [3],
        [4]], device='cuda:0')
tensor([5], device='cuda:0')
